In [23]:
import os
import shutil
import numpy as np
import random
import cv2

from cfgs.config import cfg

In [2]:
# 如果设为True，将从整个`classify_frames`文件夹读取，并将清空整个`TARGET_DATA_DIR`文件夹
is_flash = False

In [3]:
RAW_DATA_DIRS = ['classify_frames/20170828']
# 原始数据文件夹，如果不重新生成数据的话，将else部分改为新数据的文件夹
if is_flash:
    RAW_DATA_DIRS = os.listdir('classify_frames')
    RAW_DATA_DIRS = [os.path.join('classify_frames', i) for i in RAW_DATA_DIRS]

TARGET_DATA_DIR = 'data'
dataset_name = 'data'

print('\n'.join(RAW_DATA_DIRS))

classify_frames/20170828


RAW_DATA_DIR里有视频文本项目的全部数据，分为多个文件夹，以子项目命名，按流程排序

1. classify_frames: 视频文件及对应标注txt文件，将用以生成将多帧以灰度图压缩到多通道的(h,w,c)的dat文件及对应的分类，用以训练分类模型
2. segment_lines： 整页文本的图片及对应的mask图，经过处理变为小块图片、mask的dat文件、mask图，用以训练分割模型
3. recognize_sequences: 单行文本的图片文件及对应文本内容的txt文件，用以训练序列识别的模型


In [4]:
if is_flash:
    if os.path.isdir(TARGET_DATA_DIR):
        shutil.rmtree(TARGET_DATA_DIR)
    os.mkdir(TARGET_DATA_DIR)
    print('重建了data文件夹')

数据处理部分，`cv2.VideoCapture()`截取所有帧，并根据`cfg.frame_extract`生成所有的结果(shape=(h, w, num_of_frames))

In [5]:
import time

files = []

for raw_data_dir in RAW_DATA_DIRS:
    for filename in os.listdir(raw_data_dir):
        if filename.endswith('txt'):
            continue
        files.append(os.path.join(raw_data_dir, filename))
print('\n'.join(files))

classify_frames/20170828/VID_20170828_121751.mp4
classify_frames/20170828/VID_20170828_121853.mp4
classify_frames/20170828/VID_20170828_132124.mp4
classify_frames/20170828/VID_20170828_115325.mp4
classify_frames/20170828/VID_20170828_132520.mp4
classify_frames/20170828/VID_20170828_094102.mp4
classify_frames/20170828/VID_20170828_121939.mp4
classify_frames/20170828/VID_20170828_094527.mp4
classify_frames/20170828/VID_20170828_095152.mp4
classify_frames/20170828/VID_20170828_120752.mp4
classify_frames/20170828/VID_20170828_121316.mp4
classify_frames/20170828/VID_20170828_093825.mp4
classify_frames/20170828/VID_20170828_094237.mp4
classify_frames/20170828/VID_20170828_093921(1).mp4
classify_frames/20170828/VID_20170828_115412.mp4
classify_frames/20170828/VID_20170828_132208.mp4
classify_frames/20170828/VID_20170828_120458.mp4
classify_frames/20170828/VID_20170828_093739.mp4
classify_frames/20170828/VID_20170828_120844.mp4
classify_frames/20170828/VID_20170828_121038.mp4
classify_frames/2

In [9]:
# 对所有文件进行数据生成
for file in files:
    piece_cnt = 0
    frames = []
    START = time.time()
    print('读取{}……'.format(file))
    cap = cv2.VideoCapture(file)
    total_frame = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    os.mkdir(os.path.join(TARGET_DATA_DIR, '.'.join(file.split('/')[-1].split('.')[:-1])))
    
    filename = '.'.join(file.split('/')[-1].split('.')[:-1])
    extension = file.split('/')[-1].split('.')[-1]
    for i in range(total_frame):
        a, b = cap.read()
        if a:
            frame = cv2.cvtColor(b, cv2.COLOR_BGR2GRAY)
            frame = cv2.resize(frame, (224, 224))
            frames.append(frame)
            cv2.imwrite(os.path.join(TARGET_DATA_DIR, filename, '{}.png'.format(i)), frame)
    shutil.copy('.'.join(file.split('.')[:-1]) + '.txt', os.path.join(TARGET_DATA_DIR, filename, 'labels.txt'))
    END = time.time()
    print('完成了{}, 耗时{}s'.format(file, END - START))

读取classify_frames/20170828/VID_20170828_121751.mp4……
完成了classify_frames/20170828/VID_20170828_121751.mp4, 耗时5.595703601837158s
读取classify_frames/20170828/VID_20170828_121853.mp4……
完成了classify_frames/20170828/VID_20170828_121853.mp4, 耗时6.22046160697937s
读取classify_frames/20170828/VID_20170828_132124.mp4……
完成了classify_frames/20170828/VID_20170828_132124.mp4, 耗时5.534625053405762s
读取classify_frames/20170828/VID_20170828_115325.mp4……
完成了classify_frames/20170828/VID_20170828_115325.mp4, 耗时6.0824360847473145s
读取classify_frames/20170828/VID_20170828_132520.mp4……
完成了classify_frames/20170828/VID_20170828_132520.mp4, 耗时5.761683225631714s
读取classify_frames/20170828/VID_20170828_094102.mp4……
完成了classify_frames/20170828/VID_20170828_094102.mp4, 耗时5.878222227096558s
读取classify_frames/20170828/VID_20170828_121939.mp4……
完成了classify_frames/20170828/VID_20170828_121939.mp4, 耗时5.9917521476745605s
读取classify_frames/20170828/VID_20170828_094527.mp4……
完成了classify_frames/20170828/VID_20170828_094527.mp4, 耗时6.

至此为止，所有的视频文件以帧为单位 0~total_frame

重新生成records，需要遍历每个data_dir并获取label

In [26]:
records = []
videos = os.listdir(TARGET_DATA_DIR)
for video in videos:
    frames_dir = os.path.join(TARGET_DATA_DIR, video)
    frames = os.listdir(frames_dir)
    label_file = os.path.join(frames_dir, 'labels.txt')
    labels = open(label_file, 'r').readlines()[0]
    for frame in frames:
        if frame == 'labels.txt':
            continue
        frame_path = os.path.join(frames_dir, frame)
        frame_idx = int(frame.split('.')[0])
        label = labels[frame_idx]
        records.append('{} {}\n'.format(frame_path, label))

print(records)

['data/VID_20170828_120953/287.png 1\n', 'data/VID_20170828_120953/259.png 1\n', 'data/VID_20170828_120953/107.png 1\n', 'data/VID_20170828_120953/911.png 1\n', 'data/VID_20170828_120953/325.png 0\n', 'data/VID_20170828_120953/753.png 1\n', 'data/VID_20170828_120953/626.png 1\n', 'data/VID_20170828_120953/573.png 0\n', 'data/VID_20170828_120953/246.png 1\n', 'data/VID_20170828_120953/266.png 1\n', 'data/VID_20170828_120953/128.png 1\n', 'data/VID_20170828_120953/201.png 0\n', 'data/VID_20170828_120953/505.png 1\n', 'data/VID_20170828_120953/878.png 1\n', 'data/VID_20170828_120953/812.png 1\n', 'data/VID_20170828_120953/795.png 0\n', 'data/VID_20170828_120953/900.png 1\n', 'data/VID_20170828_120953/459.png 0\n', 'data/VID_20170828_120953/339.png 0\n', 'data/VID_20170828_120953/623.png 1\n', 'data/VID_20170828_120953/674.png 0\n', 'data/VID_20170828_120953/97.png 0\n', 'data/VID_20170828_120953/718.png 1\n', 'data/VID_20170828_120953/681.png 0\n', 'data/VID_20170828_120953/657.png 0\n', 

In [27]:
# 将所有数据分为训练集与测试集
random.shuffle(records)
test_ratio = 0.1
total_num = len(records)
test_num = int(test_ratio * total_num)
train_num = total_num - test_num
train_records = records[0:train_num]
test_records = records[train_num:]

In [28]:
# save to text file
all_out_file = open(dataset_name + "_all.txt", "w")
for record in records:
    all_out_file.write(record)
all_out_file.close()

train_out_file = open(dataset_name + "_train.txt", "w")
for record in train_records:
    train_out_file.write(record)
train_out_file.close()

test_out_file = open(dataset_name + "_test.txt", "w")
for record in test_records:
    test_out_file.write(record)
test_out_file.close()